In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import torch
from torch import nn
import itertools
import joblib
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import gc
tqdm.pandas()
gc.collect()

C:\Users\ricky\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


15

# load data

In [2]:
%%time
df = pd.read_feather('../data/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.feather')
df = df.loc[df.dt>=12,:] #取最近一年即可
print(df.shape,df.isnull().sum().sum())
df.head()

(19786604, 53) 1509444
Wall time: 2min 18s


,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
13188928,12.0,10115966.0,15,1.0,9678.450445,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188938,12.0,10115966.0,18,1.0,9624.614033,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188944,12.0,10115966.0,25,1.0,2338.181448,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188946,12.0,10115966.0,26,2.0,16627.150110,2.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188957,12.0,10115966.0,37,13.0,18485.113446,3.0,10.0,0.0,0.0,0.27,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0


In [3]:
test = pd.read_feather('../data/需預測的顧客名單及提交檔案範例.feather')
print(test.shape)
test.head()

(500000, 4)


,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26


In [4]:
df_groupby_chid = pd.read_feather('../data/df_groupby_chid_preprocessed.feather')
df_groupby_chid

,chid,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,...,age_2.0,age_3.0,age_4.0,age_5.0,age_6.0,age_7.0,age_8.0,age_9.0,primary_card_0.0,primary_card_1.0
0,10000000.0,-0.496861,0.773526,-0.096182,-0.435581,-0.437361,-0.432504,-0.164106,-0.633559,-0.193868,...,-0.419145,-0.621392,1.680625,-0.476502,-0.315847,-0.136068,-0.042275,-0.007616,-0.29984,0.29984
1,10000001.0,0.367163,-0.192007,-0.096182,-0.435581,0.425479,1.644711,-0.164106,-0.633559,-0.193868,...,-0.419145,-0.621392,-0.595017,2.098627,-0.315847,-0.136068,-0.042275,-0.007616,-0.29984,0.29984
2,10000002.0,-0.000899,0.219296,-0.096182,-0.435581,-0.437361,-0.432504,-0.164106,1.747482,-0.193868,...,-0.419145,1.609290,-0.595017,-0.476502,-0.315847,-0.136068,-0.042275,-0.007616,-0.29984,0.29984
3,10000003.0,1.126289,-1.040320,-0.096182,-0.435581,-0.437361,-0.432504,-0.164106,-0.633559,-0.193868,...,-0.419145,-0.621392,-0.595017,-0.476502,3.166093,-0.136068,-0.042275,-0.007616,-0.29984,0.29984
4,10000004.0,-0.902649,0.639167,-0.096182,0.797201,-0.437361,-0.432504,-0.164106,1.747482,-0.193868,...,2.385812,-0.621392,-0.595017,-0.476502,-0.315847,-0.136068,-0.042275,-0.007616,-0.29984,0.29984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,10499995.0,-2.255274,-1.040320,-0.096182,7.489447,-0.437361,-0.432504,-0.164106,-0.633559,-0.193868,...,-0.419145,-0.621392,1.680625,-0.476502,-0.315847,-0.136068,-0.042275,-0.007616,-0.29984,0.29984
499996,10499996.0,0.562695,-0.410512,-0.096182,-0.435581,-0.437361,2.450882,-0.164106,-0.633559,-0.193868,...,-0.419145,1.609290,-0.595017,-0.476502,-0.315847,-0.136068,-0.042275,-0.007616,-0.29984,0.29984
499997,10499997.0,0.416161,-0.246762,-0.096182,-0.435581,2.648710,-0.432504,-0.164106,-0.633559,-0.193868,...,-0.419145,1.609290,-0.595017,-0.476502,-0.315847,-0.136068,-0.042275,-0.007616,-0.29984,0.29984
499998,10499998.0,0.758728,-1.040320,-0.096182,0.425835,-0.437361,-0.432504,-0.164106,-0.633559,-0.193868,...,-0.419145,-0.621392,1.680625,-0.476502,-0.315847,-0.136068,-0.042275,-0.007616,-0.29984,0.29984


In [5]:
X_pca = joblib.load('../model/X_pca_for_knn.pkl')
X_pca

array([[ 1.35063364e+00, -2.66063759e-02, -3.26480326e-01, ...,
        -1.49153345e-01, -4.83359140e-02,  6.66182595e-02],
       [-5.22890190e-01, -1.65049606e-02, -3.26696867e+00, ...,
        -1.19622552e-01,  4.83849501e-01, -3.59875796e-01],
       [-5.88312618e-01, -7.90730018e-04,  1.02323246e+00, ...,
        -4.79204579e-01,  2.65520518e-01,  6.75998172e-01],
       ...,
       [ 5.48692032e-01, -9.58478302e-03, -1.83449548e-01, ...,
         3.49879436e-01,  1.87853899e-01, -1.29514056e-01],
       [-1.61025541e-01, -1.54671756e-02,  1.08035440e+00, ...,
        -1.32240077e-01,  1.00498809e-01,  6.23435482e-01],
       [-2.42410965e+00,  6.33518384e-03, -3.67732680e-03, ...,
         9.25123915e-02,  1.18056191e-01,  7.98769461e-02]])

# define functions

In [6]:
chid = test['chid'].values[0]

def chid2idx(chid:int)->int:
    '''
    chid到idx的映射
    '''
    return df_groupby_chid[df_groupby_chid.chid==chid].index[0]
    
def get_values_counts(chid:int)->dict:
    '''
    chid到shop_tag之value_counts的映射
    '''
    n_class = 49
    d = dict(zip([str(i) for i in range(1,n_class)]+['other'],[0 for _ in range(n_class)]))
    frame = df.loc[df.chid==chid,'shop_tag'].value_counts()
    for shop_tag,counts in zip(frame.index,frame.values):
        d[shop_tag] = counts
    return d

def get_probs(value_counts:dict)->np.array:
    '''
    將value_counts(字典)轉成機率向量(np.array)
    '''
    counts = np.array(list(value_counts.values()))
    probs = counts / np.sum(counts)
    return probs

def initial_h0(probs:np.array)->torch.FloatTensor:
    '''
    初始化rnn的隱變量h0
    '''
    return torch.FloatTensor([probs])

def get_answer(value_counts:dict)->list:
    '''
    輸入value_counts 49種shop_tag的機率
    輸出answer [top1,top2,top3]
    '''
    官方指認欄位 = ['2','6','10','12','13','15','18','19','21','22','25','26','36','37','39','48']
    value_counts = dict((k,value_counts[k]) for k in value_counts.keys() if k in 官方指認欄位) # 將官方欄位裡面沒有的濾掉(保留在官方指定欄位的)
    L = sorted(value_counts.items(),key=lambda item:item[1],reverse=True) # 根據value做排序
    L = L[:3] # 取前三
    L = [i for i in L if i[1] > 0]
    L = [tup[0] for tup in L] # 只取key(shop_tag)
    return L

print('chid:',chid2idx(chid))
print('values_counts:',get_values_counts(chid))
print('probs:',get_probs(get_values_counts(chid)))
print('h0:',initial_h0(get_probs(get_values_counts(chid))))
print('answer:',get_answer(get_values_counts(chid)))

chid: 128239
values_counts: {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0, '18': 0, '19': 0, '20': 0, '21': 0, '22': 0, '23': 0, '24': 0, '25': 1, '26': 0, '27': 0, '28': 0, '29': 0, '30': 0, '31': 0, '32': 0, '33': 0, '34': 0, '35': 0, '36': 0, '37': 3, '38': 0, '39': 0, '40': 0, '41': 0, '42': 0, '43': 0, '44': 0, '45': 1, '46': 0, '47': 0, '48': 0, 'other': 3}
probs: [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.125 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.375 0.    0.    0.    0.    0.    0.    0.    0.125 0.    0.    0.
 0.375]
h0: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1250, 0.0000, 0.

# build RNN

In [7]:

class R2D2(nn.Module):
  def __init__(self,input_size,output_size,hidden_size):
    super().__init__()
    self.l1 = nn.Linear(input_size,hidden_size) #layer1線性層
    self.l2 = nn.GRUCell(hidden_size,output_size) #layer2是GRU
  def forward(self,x,hx):
    features =  torch.tanh(self.l1(x)) #將x再經過layer1抽取有用的特徵稱之為features
    logits = self.l2(features,hx) #將features(有用的特徵)和hx(每個shop_tag的概率)一併丟給layer2計算出logits(輸出)
    return torch.softmax(logits,dim=1) # 將logits輸出轉換成機率(softmax)
rnn = R2D2(3,3,3)
rnn

R2D2(
  (l1): Linear(in_features=3, out_features=3, bias=True)
  (l2): GRUCell(3, 3)
)

# 預測函數

In [8]:
def predict_function(chid:int)->list:
    '''
    輸入:chid(int)
    輸出:[top1,top2,top3](list of string)
    '''
    官方指認欄位 = ['2','6','10','12','13','15','18','19','21','22','25','26','36','37','39','48']
    idx = chid2idx(chid) # 取得這個chid的索引
    values_counts = get_values_counts(chid) # 取得這個chid之shop_tag的value_counts
    probs = get_probs(values_counts) # 將value_counts轉換成機率
    hx = initial_h0(probs) # 初始化h0
    answer = get_answer(values_counts) # 取得基於規則的答案
    remain = 3-len(answer) # 計算剩餘多少要給模型做填補
    input = X_pca[[idx]] # 這個chid的pca特徵
    input_size = input.shape[1] # 模型輸入維度
    output_size = 49 # 輸出層維度
    hidden_size = output_size # 隱藏層維度
    rnn = R2D2(input_size,hidden_size,output_size) # 實例化模型
    for _ in range(remain): # 遍歷剩餘數量做填補
        print('觸發rnn')
        hx = rnn(torch.FloatTensor(input),hx) # 輸入(特徵加上概率)輸出(概率tensor)
        probs = dict(zip([ str(i) for i in range(1,49)]+['other'],hx.ravel().detach().numpy())) # 概率tensor轉換成概率字典
        probs = dict((k,probs[k]) for k in probs.keys() if k not in answer) # 將answer裡面已經有的濾掉(保留不在answer裡的)
        probs = dict((k,probs[k]) for k in probs.keys() if k in 官方指認欄位) # 將官方欄位裡面沒有的濾掉(保留在官方指定欄位的)
        shop_tag = max(probs,key=probs.get) # 取得概率字典中概率最大value(機率)對應的key(shop_tag)
        answer.append(str(shop_tag)) #將該shop_tag併入到answer
    assert (len(answer) == 3) and (len(np.unique(answer)) == 3)# 確認有三個不一樣的shop_tag
    return answer 

# 測試預測函數

In [9]:
submit = []
for _ in tqdm(range(30)):
    chid = np.random.choice(test['chid'].values)
    submit.append(predict_function(chid))
submit[:10]

 33%|███▎      | 10/30 [00:00<00:00, 38.87it/s]

觸發rnn
觸發rnn
觸發rnn


 73%|███████▎  | 22/30 [00:00<00:00, 45.15it/s]

觸發rnn
觸發rnn
觸發rnn
觸發rnn
觸發rnn
觸發rnn


100%|██████████| 30/30 [00:00<00:00, 49.19it/s]


[['37', '10', '2'],
 ['2', '10', '18'],
 ['37', '2', '13'],
 ['36', '37', '48'],
 ['48', '6', '19'],
 ['6', '10', '36'],
 ['36', '37', '26'],
 ['12', '13', '39'],
 ['2', '37', '15'],
 ['37', '15', '2']]